In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime, date
from datetime import datetime, timedelta
import riskfolio as pf
import numpy as np
import statsmodels.api as sm
pd.options.mode.chained_assignment = None 

In [ ]:
risk_free_rate = 0.0538

In [ ]:
df = pd.read_csv('/...choose your path...data.csv',header=0, index_col=0)

## 6. Find the variance – covariance and correlation matrixes

In [ ]:
# Calculate the variance-covariance matrix
var_cov_matrix = df.cov()

# Calculate the correlation matrix
corr_matrix = df.corr()

## 7. With no constraints whatsoever, find the Global Minimum Variance Portfolio (GMVP) TBU

In [ ]:
df1=df.copy()

# Calculate the average of 'TLT' and 'GC=F'
df1['A'] = df1[['TLT', 'GC=F']].mean(axis=1)

# Calculate the average of all other columns except 'TLT' and 'GC=F'
other_columns = df1[df1.columns.difference(['TLT', 'GC=F'])]
df1['B'] = other_columns.mean(axis=1)

result_df = df1[['A', 'B']]

In [ ]:
cov_matrix = df1.cov()

# Extract the necessary statistics
sigma_A_squared = cov_matrix.loc['A', 'A']
sigma_AB = cov_matrix.loc['A', 'B']

# Calculate beta_B
beta_B = sigma_AB / sigma_A_squared

# Ratio of weights for asset A and B
w_A_over_w_B = -beta_B

w_A = 0.3 #change to whatever weight you suggest
w_B = w_A / w_A_over_w_B

## 8. Constraint 1: assuming now that the portfolio is fully invested (the sum of all weight- ings must be equal to 1, with short positions allowed) find the GMV of your portfolio specifying asset weightings, standard deviation and return


In [ ]:

# Building the portfolio object
port = pf.Portfolio(returns=df)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Configuring short weights options

port.sht = True # Allows to use Short Weights

port.uppersht =  10  # Maximum value of sum of short weights in absolute value
port.upperlng = 11 # Maximum value of sum of positive weights
port.budget = 1.0 # port.upperlng - port.uppersht

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

In [ ]:
# Calculate the covariance matrix of the returns (annualized)
cov_matrix_annual = df.cov() * 252  # Assuming 252 trading days in a year

# Calculate annualized average returns
avg_returns_annual = df.mean() * 252

# Initialize a dictionary to store the results
portfolio_metrics_8 = {}

weights = w['weights'].values
# Portfolio return is the weighted sum of individual expected returns (annualized)
portfolio_return = np.dot(weights, avg_returns_annual)
# Portfolio variance is a quadratic form of the weights and the covariance matrix (annualized)
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
# Portfolio standard deviation is the square root of variance (annualized)
portfolio_std_dev = np.sqrt(portfolio_variance)
# Sharpe ratio is the annualized excess return over risk-free rate divided by the standard deviation
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev
    
# Store the results
portfolio_metrics_8['a'] = {
    'Return': portfolio_return,
    'Standard Deviation': portfolio_std_dev,
    'Sharpe Ratio': sharpe_ratio
    }

portfolio_metrics_8

## 9. Constraint 2: define three minimum required return levels along the efficient frontier

(c) Find the GMVP indicating its standard deviation and expected return. Compare with GMVP identified in 7 and 8


In [ ]:
# Building the portfolio object
port = pf.Portfolio(returns=df)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estimate optimal portfolio:
model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'MinRisk' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w1 = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

In [ ]:
cov_matrix_annual = df.cov() * 252  # Assuming 252 trading days in a year
avg_returns_annual = df.mean() * 252
portfolio_metrics_9 = {}

weights = w1['weights'].values
portfolio_return = np.dot(weights, avg_returns_annual)
portfolio_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
portfolio_std_dev = np.sqrt(portfolio_variance)
sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev    

portfolio_metrics_9['a'] = {
    'Return': portfolio_return,
    'Standard Deviation': portfolio_std_dev,
    'Sharpe Ratio': sharpe_ratio
    }


(b) Find the efficient frontier

In [ ]:
points = 300 # Number of points of the frontier
frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

In [ ]:
# Plotting the efficient frontier
label = 'Max Risk Adjusted Return Portfolio' # Title of plot
mu = port.mu # Expected returns
cov = port.cov # Covariance matrix
returns = port.returns # Returns of the assets

ax = pf.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, cmap='viridis', s=16, c='r', height=6, width=10, ax=None)

(a) Specify the weights of each asset class for each of the three portfolios

In [ ]:
cov_matrix_annual = df.cov() * 252  # Assuming 252 trading days in a year
avg_returns_annual = df.mean() * 252
portfolio_metrics2 = {}

for column in frontier.columns:
    weights = frontier[column].values
    portfolio_return = np.dot(weights, avg_returns_annual)
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

    portfolio_metrics2[column] = {
        'Return': portfolio_return,
        'Standard Deviation': portfolio_std_dev,
        'Sharpe Ratio': sharpe_ratio
    }

In [ ]:
desired_returns = [0.15, 0.25, 0.30] #Define desired returns for portfolios
matching_portfolios = {}

for return_value in desired_returns:
    closest_portfolio = min(portfolio_metrics2.items(), key=lambda x: abs(x[1]['Return'] - return_value))
    matching_portfolios[f"Closest to {return_value}"] = closest_portfolio

In [ ]:
portfolio_numbers = [info[0] for info in matching_portfolios.values()]

three = frontier[portfolio_numbers]

In [ ]:
cov_matrix_annual = df.cov() * 252  # Assuming 252 trading days in a year

avg_returns_annual = df.mean() * 252

portfolio_metrics = {}

for column in three.columns:
    weights = three[column].values
    portfolio_return = np.dot(weights, avg_returns_annual)
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
    portfolio_std_dev = np.sqrt(portfolio_variance)
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std_dev

    portfolio_metrics[column] = {
        'Return': portfolio_return,
        'Standard Deviation': portfolio_std_dev,
        'Sharpe Ratio': sharpe_ratio
    }

## 10. Constraint 3: adding now a Risk-Free asset

In [ ]:
# Identifying the portfolio with the highest Sharpe Ratio
highest_sharpe = max(portfolio_metrics2, key=lambda k: portfolio_metrics2[k]['Sharpe Ratio'])

# Portfolio with the highest Sharpe Ratio and its metrics
highest_sharpe_portfolio = portfolio_metrics2[highest_sharpe]
highest_sharpe, highest_sharpe_portfolio

w_market = frontier[highest_sharpe]

In [ ]:
market_portfolio_weights = np.array(w_market)

avg_returns_annual = np.array(avg_returns_annual)

# Calculate market portfolio return and standard deviation
market_portfolio_return = np.dot(market_portfolio_weights, avg_returns_annual)
market_portfolio_std_dev = np.sqrt(np.dot(market_portfolio_weights.T, 
                                          np.dot(cov_matrix_annual, market_portfolio_weights)))

# Calculate the Sharpe ratio for the market portfolio
market_portfolio_sharpe_ratio = (market_portfolio_return - risk_free_rate) / market_portfolio_std_dev

# Slope of the Capital Market Line (CML) is the Sharpe ratio of the market portfolio
cml_slope = market_portfolio_sharpe_ratio

# Generate a range of standard deviations (volatility levels)
std_dev_range = np.linspace(0, market_portfolio_std_dev * 2, 100)

# Calculate the expected returns on the CML
cml_returns = risk_free_rate + (cml_slope * std_dev_range)

plt.plot(std_dev_range, cml_returns, label='Capital Market Line')
plt.scatter(market_portfolio_std_dev, market_portfolio_return, color='red', label='Market Portfolio')
plt.xlabel('Standard Deviation (Risk)')
plt.ylabel('Expected Return')
plt.title('Capital Market Line (CML)')
plt.legend()
plt.grid(False)
plt.show()

## 11. Take the same three portfolios

In [ ]:
ps = pd.DataFrame(portfolio_metrics).T

three_returns = ps['Return']
three_returns = np.array(three_returns)

three_std = ps['Standard Deviation']
three_std = np.array(three_std)

In [ ]:
# Calculate Sharpe Ratios for the three original portfolios
three_sharpe_ratios = (three_returns - risk_free_rate) / three_std

weights_market = three_std / market_portfolio_std_dev

# Calculate their expected returns based on the CML
expected_returns_cml = risk_free_rate + weights_market * (market_portfolio_return - risk_free_rate)

# The standard deviations along the CML are the same as the original portfolios
# since we've used the same volatility to calculate the weights on the market portfolio
std_dev_cml = three_std

# Calculate Sharpe Ratios for the three portfolios along the CML
sharpe_ratios_cml = (expected_returns_cml - risk_free_rate) / std_dev_cml

# Results
portfolio_weights_cml = weights_market
portfolios_cml_metrics = {
    'Expected Returns': expected_returns_cml,
    'Standard Deviations': std_dev_cml,
    'Sharpe Ratios': sharpe_ratios_cml
}

# Output the results
three_sharpe_ratios, portfolio_weights_cml, portfolios_cml_metrics, market_portfolio_sharpe_ratio

In [ ]:
1 - portfolio_weights_cml # rf_weights

## 12. Define the position on the SML of the three portfolios (CAPM framework)

In [ ]:
three1 = three.T

In [ ]:
# Calculate the returns of the market portfolio
market_portfolio_returns = df.dot(market_portfolio_weights)

# Calculate the returns of the three portfolios in 'three1'
# Note: 'three1' should be defined as a DataFrame with portfolio weights before this step.
three_portfolios_returns = df.dot(three1.T)

three_portfolios_returns['M'] = market_portfolio_returns
portfolio_market_cov = three_portfolios_returns.cov()*252
portfolio_market_cov = portfolio_market_cov['M'][:-1]

In [ ]:
# Assuming 'three_portfolios_returns' is your DataFrame
market_returns = three_portfolios_returns.iloc[:, 3]  # The fourth column as market returns, zero-based indexing
portfolios = three_portfolios_returns.columns[:3]  # Selecting the first three columns as portfolio names

results = {}

for portfolio in portfolios:
    # Preparing the data
    y = three_portfolios_returns[portfolio]  # Portfolio returns
    X = sm.add_constant(market_returns)  # Adding a constant for the regression

    # Running the regression
    model = sm.OLS(y, X).fit()

    # Storing alphas, betas and checking alpha significance
    alpha, beta = model.params['const'], model.params[market_returns.name]
    p_value_alpha = model.pvalues['const']  # P-value for alpha

    # Storing results
    results[portfolio] = {
        'beta': beta,
        'alpha': alpha,
        'p-value': p_value_alpha,
        'alpha_significant': p_value_alpha < 0.05  # Typically, a p-value < 0.05 implies significance
    }